In [1]:
import numpy as np
import pandas as pd
import os
from modules.Date2 import find_starting_point, find_specific_date, convert_date
from modules.variables2 import *
from modules.Data_Quality import *
from modules.Cost_centre_structure import *

In [31]:
end_month = 'Apr'
end_year = '2021/22'

files_info =[]
dataframes = []

root = './'
input_folder = root+'Input files/' + 'forecast_12_May_2021_input'
output_folder = root+'Prepared Files/' + 'Forecasts_12_Month_May_21'
combined_folder = root+'Combined files/' +'Forecasts_12_Month_May_21_Combined/'
directory = os.fsencode(input_folder)
num = 0

In [32]:
def process_file(file_path, division):
    global num
    xls = pd.ExcelFile(file_path)
    subregion_dict = division_to_subregion.get(division)

    if not subregion_dict:
        print(f"No subregion dictionary found for division {division}")
        return

    for sheet_name in xls.sheet_names:
        if any(sheet_name in cost_centres for cost_centres in subregion_dict.values()):
            num += 1
            # Load the data from the sheet, specifying no header to properly identify the row
            data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
            print(division, sheet_name)
            start_month, start_year, row, start_col = find_starting_point(data)
            end_row, end_col = find_specific_date(data, end_month, end_year)
            dates = convert_date(data, row, start_col, end_col)

            extracted_data = extract_var_columns(data, start_col, end_col)
            extracted_data, region, division, cost_centre = get_metadata(extracted_data, division, sheet_name)
            final_df = pd.DataFrame({'Date': dates})
            final_df = pd.concat([final_df, extracted_data.reset_index(drop=True)], axis=1)
            num_col = len(final_df.columns)
            has_missing, rows_with_missing = check_missing_values(final_df, columns_to_keep)
            has_secured_missing = check_missing_secured(final_df)
            duplicated = check_duplicates(final_df, columns_to_keep)
            #incorrect_relationship = check_relationships(final_df)

            var_name = f"{os.path.basename(file_path)}_{sheet_name}_missing"
            globals()[var_name] = rows_with_missing

            file_info = {'file_name': os.path.basename(file_path).split('.')[0],
                         'num': num,
                         'region': region,
                         'division': division,
                         'cost_centre': cost_centre,
                         'num_col': num_col,
                         'Start month': start_month,
                         'Start year': start_year,
                         'row': row,
                         'End column': end_col,
                         'missing': has_missing,
                         'secured missing': has_secured_missing,
                         'duplicated': duplicated,
                         }

            files_info.append(file_info)
            dataframes.append(final_df)
            output_file = f"{division}_{sheet_name.replace(' ', '_')}"
            prepared_file_path = os.path.join(output_folder, f'{output_file}_ok.csv')
            final_df.to_csv(prepared_file_path, index=False)

In [33]:
# Main loop to process all files
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") or filename.endswith(".xls") or filename.endswith(".xlsm"):
        file_path = os.path.join(input_folder, filename)

        # Extract division from filename
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        name_parts = base_name.split('-')
        if len(name_parts) > 1:
            division = name_parts[1].strip().lower()
            process_file(file_path, division)

files_info_df = pd.DataFrame(files_info)
files_info_df.to_csv('files_info_fcst21_1.csv', index=False)

advisory AFM
advisory Proc
advisory Disp
advisory SHQTel
advisory Suiko
amcl Group
amcl AMA
amcl Vertex
cost management Edinburgh
cost management Glasgow
cost management Belfast
cost management Leeds
cost management NorthWest
cost management Sheffield
cost management Birmingham
cost management Nottingham
cost management Bristol
cost management Cambridge
cost management CLGov
cost management HiTech
cost management HE
cost management Occupier
cost management Developer
cost management Co
infrastructure CM Scotland
infrastructure PM Scotland
infrastructure PMO Scotland
infrastructure CM NW
infrastructure PM NW
infrastructure PMO NW
infrastructure CM YNE
infrastructure PM YNE
infrastructure PMO YNE
infrastructure CM SE
infrastructure PM SE
infrastructure PMO SE
infrastructure MOD
infrastructure PA
infrastructure Co
project management Edinburgh
project management Glasgow
project management Belfast
project management Leeds
project management North West
project management Project Controls
proj

In [34]:
len(dataframes)

58

In [37]:
# Filter and combine DataFrames to include only 'Advisory' and 'AMCL' divisions
included_divisions = ['advisory', 'amcl']

filtered_dataframes1 = [
    df for df in dataframes
    if df['Division'].iloc[0].lower() in included_divisions
]

In [35]:
excluded_divisions = ['advisory', 'amcl']  # Add the names of divisions to exclude
excluded_cost_centres = ['PM_RE_Digital', 'CM_alinea', 'INF_Def_Nth', 'INF_Def_SE','INF_T&U_SW',
                         'INF_Def_CM_SW','INF_Def_P3M_SW', 'INF_Digital', 'PM_Pcon_Sth', 'PM_Co', 'ADV_SHQUK', 'ADV_SHQTel', 'ADV_R&H', 'AMCL_UK&I', 'AMCL_IES', 'ADV_NZ', 'CM_Co']  # Add the names of cost centres to exclude

# Filter dataframes to exclude problematic divisions and cost centres
filtered_dataframes = []
for df in dataframes:
    if df['Division'].iloc[0].lower() not in excluded_divisions and df['Cost_Centre'].iloc[0] not in excluded_cost_centres:
        filtered_dataframes.append(df)

In [36]:
len(filtered_dataframes)

47

In [37]:
combined_df = pd.concat(filtered_dataframes, ignore_index=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1584 non-null   datetime64[ns]
 1   secured               1551 non-null   object        
 2   probable              994 non-null    object        
 3   possible              981 non-null    object        
 4   speculative           974 non-null    object        
 5   biso                  0 non-null      object        
 6   budget_income         1344 non-null   object        
 7   other_direct_costs    1584 non-null   float64       
 8   direct_costs          1584 non-null   float64       
 9   gross_profit          1584 non-null   object        
 10  budget_gross_profit   0 non-null      object        
 11  overheads             1019 non-null   float64       
 12  subcon                1180 non-null   object        
 13  net_profit        

In [38]:
# Define columns to keep and columns to rename
# Calculate `income_ex_biso` if it's not already calculated
combined_df['income_ex'] = combined_df[['secured', 'probable', 'possible', 'speculative']].sum(axis=1, skipna=True)
    
columns_to_keep = [
    'Date','income_ex',  'direct_costs', 'overheads',
     'total_headcount', 'Region', 'Cost_Centre', 'Division' 
]

combined_df = combined_df[columns_to_keep]

cols_not_rename = ['Region', 'Cost_Centre', 'Division', 'Date']
# Create a list of columns to rename (exclude columns_to_keep and 'Region', 'Cost_Centre', 'Division', 'Date')
columns_to_rename = [col for col in combined_df.columns if col not in cols_not_rename]

# Dictionary to map old names to new names
rename_mapping = {col: col + '_fcst12' for col in columns_to_rename}

# Rename columns in the dataframe
combined_df_fcst = combined_df.rename(columns=rename_mapping)

In [11]:
combined_df_fcst[combined_df_fcst.isnull().any(axis=1)]

,Date,income_ex_fcst12,direct_costs_fcst12,overheads_fcst12,total_headcount_fcst12,Region,Cost_Centre,Division
383,2022-04-01,90,-0.000000,NaN,0,Co,CM_Co,Cost management
384,2019-05-01,238.018723,158.241473,NaN,30.589821,Scotland,INF_CM_Scotland,Infrastructure
385,2019-06-01,205.05617,159.546958,NaN,31.404821,Scotland,INF_CM_Scotland,Infrastructure
386,2019-07-01,223.850162,173.762462,NaN,31.404821,Scotland,INF_CM_Scotland,Infrastructure
387,2019-08-01,235.207672,169.246930,NaN,30.404821,Scotland,INF_CM_Scotland,Infrastructure
...,...,...,...,...,...,...,...,...
919,2021-12-01,0,104.641912,NaN,5.65,Co,INF_Co,Infrastructure
920,2022-01-01,0,104.641912,NaN,5.65,Co,INF_Co,Infrastructure
921,2022-02-01,0,104.641912,NaN,5.65,Co,INF_Co,Infrastructure
922,2022-03-01,0,104.641912,NaN,5.65,Co,INF_Co,Infrastructure


In [40]:
# Read the CSV file
Folder = root
combined_file_path = Folder + 'hist_df_all.csv'
combined_file_df = pd.read_csv(combined_file_path)

# Ensure date columns are in datetime format
combined_file_df['Date'] = pd.to_datetime(combined_file_df['Date'], format='%Y-%m-%d')
#combined_df_fcst['Date'] = pd.to_datetime(combined_df_fcst['Date'], format='%d/%m/%Y')

In [41]:


# Define the date range for the new_df
start_date = '2021-05-01'
end_date = '2022-04-01'

#start_date = '01/05/2021'
#end_date = '01/04/2022'

# Filter new_df for the specified date range
new_df_filtered = combined_df_fcst[
    (combined_df_fcst['Date'] >= start_date) & (combined_df_fcst['Date'] <= end_date)
    ]

# List of columns to add
columns_to_add = ['income_ex_fcst12', 'direct_costs_fcst12', 'overheads_fcst12', 'total_headcount_fcst12']

In [15]:
new_df_filtered

,Date,income_ex_fcst12,direct_costs_fcst12,overheads_fcst12,total_headcount_fcst12,Region,Cost_Centre,Division
12,2021-05-01,48.3,41.026588,21.478890,5.128011,other_regions,CM_Edinburgh,Cost management
13,2021-06-01,53.275,45.868843,21.525166,6.128011,other_regions,CM_Edinburgh,Cost management
14,2021-07-01,65.532,53.680431,21.445744,8.128011,other_regions,CM_Edinburgh,Cost management
15,2021-08-01,80,58.583158,21.348030,9.128011,other_regions,CM_Edinburgh,Cost management
16,2021-09-01,78.937175,61.226196,21.442788,10.128011,other_regions,CM_Edinburgh,Cost management
...,...,...,...,...,...,...,...,...
1327,2021-12-01,330,225.257388,54.476623,31.174109,London,PM_PMO,Project management
1328,2022-01-01,330,229.935710,53.640218,32.174109,London,PM_PMO,Project management
1329,2022-02-01,330,226.624188,53.378883,31.174109,London,PM_PMO,Project management
1330,2022-03-01,330,226.624188,53.374757,31.174109,London,PM_PMO,Project management


In [42]:
# Add new columns to combined_file_df with NaN values initially if not already present
for column in columns_to_add:
    if column not in combined_file_df.columns:
        combined_file_df[column] = pd.NA

# Fill in the new columns with values from new_df_filtered
for index, row in combined_file_df.iterrows():
    matching_rows = new_df_filtered[
        (new_df_filtered['Cost_Centre'] == row['Cost_Centre']) &
        (new_df_filtered['Date'] == row['Date'])
        ]
    if not matching_rows.empty:
        for column in columns_to_add:
            combined_file_df.at[index, column] = matching_rows.iloc[0][column]

# Display the merged dataframe
print(combined_file_df)

           Date  income_ex  direct_costs  overheads  total_headcount  \
0    2018-05-01  313.78214    244.088380   53.18396        31.850000   
1    2018-06-01  317.64346    247.639130   55.06331        30.600000   
2    2018-07-01  328.53530    264.526980   59.65837        31.920000   
3    2018-08-01  334.38394    254.934530   49.22757        29.400000   
4    2018-09-01  326.91636    236.313300   50.95490        29.350000   
...         ...        ...           ...        ...              ...   
4279 2023-12-01  486.05929    368.813240   53.15519        30.866625   
4280 2024-01-01  410.06973    227.925930   50.51012        31.386667   
4281 2024-02-01  502.71161    245.408020   49.87197        32.330000   
4282 2024-03-01  411.60016    281.374660   45.94258        33.146667   
4283 2024-04-01  315.00000    226.326125   48.19027        30.300000   

          Region   Cost_Centre            Division  income_ex_fcst12  \
0     Consulting       ADV_AFM            Advisory             

In [56]:
new_df_filtered[new_df_filtered.isnull().any(axis=1)]
new_df_filtered.to_csv('asfasdf.csv', date_format='%d-%m-%y', index=False)

In [4]:
Folder = root + 'Combined files/'
combined_file_path = Folder + 'combined_file_fcst2021added.csv'
combined_file_df = pd.read_csv(combined_file_path)

In [43]:
combined_file_df = combined_file_df.drop(columns = ['Unnamed: 0'])

KeyError: "['Unnamed: 0'] not found in axis"

In [44]:
# Save the combined dataframe to a CSV file
combined_csv_path = combined_folder + 'hist_fcst_all.csv'
combined_file_df.to_csv(combined_csv_path, date_format='%Y-%m-%d', index=False)

excluded_dataframes = [df for df in dataframes if df['Division'].iloc[0] in excluded_divisions or df['Cost_Centre'].iloc[0] in excluded_cost_centres]
for df in excluded_dataframes:
    print(f"Excluded Division: {df['Division'].iloc[0]}, Cost Centre: {df['Cost_Centre'].iloc[0]}")

Excluded Division: Advisory, Cost Centre: ADV_SHQTel
Excluded Division: Cost management, Cost Centre: CM_Co
Excluded Division: Project management, Cost Centre: PM_Pcon_Sth
Excluded Division: Project management, Cost Centre: PM_Co
